In [1]:
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.mobilenet import MobileNet
from keras.applications.densenet import DenseNet121
from keras.applications.nasnet import NASNetLarge
from keras.applications.mobilenet_v2 import MobileNetV2
import keras
from keras import optimizers 


from keras.layers import Conv2D, GlobalMaxPooling2D, MaxPooling2D, Dense, Input, Flatten, Dropout, BatchNormalization
from keras.models import Model, load_model, Sequential
import tensorflow as tf
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns 
import numpy as np

Using TensorFlow backend.


In [2]:
data_dir = 'data/'
OUTPUT_FOLDER = "./savedRuns/"

In [3]:
labels = []
import os
for folder in os.listdir(data_dir):
    if folder != ".DS_Store":
        labels.append(folder)
        print(folder)

no person
person


In [4]:
BATCH_SIZE = 10 #150
IMAGE_SIZE=(150,150)
IMAGE_SHAPE=(150,150,3)
#Dataagumentation for better results on small data
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
                                   shear_range=0.2,
                                   zoom_range=0.3,
                                   rotation_range=50,
                                   horizontal_flip=True,
                                   fill_mode="nearest",
                                   validation_split=0.20) 
                                   
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size = IMAGE_SIZE,
    shuffle=True,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training') 

validation_generator = train_datagen.flow_from_directory(
    data_dir, 
    target_size = IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle= False,
    class_mode='categorical',
    subset='validation') 

Found 128 images belonging to 2 classes.
Found 31 images belonging to 2 classes.


In [5]:
train_generator.class_indices

{'no person': 0, 'person': 1}

In [6]:
#learningcurve plot
def history(history, show_val = False):
    plt.figure(figsize = (8,8))
    plt.plot(history.history['accuracy'], label='training accuracy')
    if show_val:
        plt.plot(history.history['val_accuracy'], label='val accuracy')
        plt.plot(history.history['val_loss'], label='val loss')
    plt.ylim(ymax=2,ymin=0)
    plt.plot(history.history['loss'], label='training loss')
    plt.title('Loss/Accuracy')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.legend()
#    plt.savefig("CNN_30_Epochs")
    plt.show()

In [7]:
#Define outputfile

filepath= "./savedRuns/BildTagging-{epoch:02d}-{val_acc:.4f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only=False)
callbacks_list = [checkpoint]

In [8]:
#Create Model    
pretrained_model = VGG16(include_top=False, weights='imagenet',input_shape=IMAGE_SHAPE)

output = pretrained_model.layers[-1].output
output = keras.layers.Flatten()(output)
pretrained_model = Model(pretrained_model.input,output)

pretrained_model.trainable = True
set_trainable = False
for layer in pretrained_model.layers:
    layer.trainable = False

for layer in pretrained_model.layers: 
    if layer.name in ['block5_conv1', 'block4_conv1']: 
        set_trainable = True 
    if set_trainable: 
        layer.trainable = True 
    else: 
        layer.trainable = False 

model = Sequential()
model.add(pretrained_model)
model.add(Dense(512,activation='relu',input_dim=pretrained_model.output_shape[1]))
model.add(Dropout(0.3))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['accuracy'])
          
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 8192)              14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)      

In [9]:
import pandas as pd 
pd.set_option('max_colwidth', -1) 
 
layers = [(layer, layer.name, layer.trainable) for layer in model.layers] 
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer  Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.training.Model object at 0x000001B7A07E9B00>,model_1,True
1,<keras.layers.core.Dense object at 0x000001B7A07E9C88>,dense_1,True
2,<keras.layers.core.Dropout object at 0x000001B7A4067B70>,dropout_1,True
3,<keras.layers.core.Dense object at 0x000001B7A5062BA8>,dense_2,True
4,<keras.layers.core.Dropout object at 0x000001B7A08BDC88>,dropout_2,True
5,<keras.layers.core.Dense object at 0x000001B7A08E38D0>,dense_3,True


In [10]:
EPOCHS = 20
hist = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples//BATCH_SIZE,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples//BATCH_SIZE,
    epochs = EPOCHS,
    verbose=1,
    callbacks=callbacks_list)
model.save('VGG_finetuning_run_30_with_pre.h5')
#history(hist, True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
12/12 [==============================] - 66s 5s/step - loss: 2.6661 - acc: 0.7417 - val_loss: 1.9541e-05 - val_acc: 1.0000

Epoch 00001: val_acc improved from -inf to 1.00000, saving model to ./savedRuns/BildTagging-01-1.0000.h5
Epoch 2/20
12/12 [==============================] - 62s 5s/step - loss: 0.9694 - acc: 0.8395 - val_loss: 2.4449e-05 - val_acc: 1.0000

Epoch 00002: val_acc did not improve from 1.00000
Epoch 3/20
12/12 [==============================] - 60s 5s/step - loss: 2.4502 - acc: 0.8228 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00003: val_acc did not improve from 1.00000
Epoch 4/20
12/12 [==============================] - 60s 5s/step - loss: 0.2816 - acc: 0.9750 - val_loss: 1.1921e-07 - val_acc: 1.0000

Epoch 00004: val_acc did not improve from 1.00000
Epoch 5/20
12/12 [==============================] - 64s 5s/step - loss: 0.9777 - acc: 0.9249 - val_loss: 5.6982 - val_acc: 0.6000

Epoch 00005: val_acc did n

In [12]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [ ]:
#Confusion Matrix
validation_generator.reset
Y_pred = model.predict_generator(validation_generator, validation_generator.samples // BATCH_SIZE+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
print(classification_report(validation_generator.classes, y_pred))

In [ ]:
ax= plt.subplot()
sns.heatmap(confusion_matrix(validation_generator.classes, y_pred), annot=True, ax = ax, fmt='g', cmap='Greens') #annot=True to annotate cells

ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels') 
ax.set_title('Confusion Matrix') 


ax.xaxis.set_ticklabels(labels,rotation=45)
ax.yaxis.set_ticklabels(labels,rotation=0)